In [ ]:
import requests
import json
import pandas as pd
import numpy as np

nhl_url = 'https://statsapi.web.nhl.com/api/v1/'

def nhl_request(path, **kwargs):
    res = requests.get(nhl_url+path, **kwargs)
    if res.status_code == 200:
        return res.json()

In [ ]:
params= {}
params['season'] = '20182019'
params['gameType'] = 'R'
response = nhl_request(f'schedule', params=params)

In [ ]:
dateArray = []
gameIDArray = []
for dateObj in response['dates']:
    date = dateObj['date']
    dateArray.append(date)
    for gameObj in dateObj['games']:
        gameIDArray.append(gameObj['gamePk'])

In [ ]:
def getSeasonGameIDs(season):
    params= {}
    params['season'] = season
    params['gameType'] = 'R'
    response = nhl_request(f'schedule', params=params)
    gameIDArray = []
    for dateObj in response['dates']:
        date = dateObj['date']
        dateArray.append(date)
        for gameObj in dateObj['games']:
            gameIDArray.append(gameObj['gamePk'])
    return gameIDArray

In [ ]:
def getGameData(gameID):
    events = []
    gameRes = nhl_request(f'game/{gameID}/feed/live')
    homeTeam = gameRes['gameData']['teams']['home']['name']
    homeId = gameRes['gameData']['teams']['home']['id']
    awayTeam = gameRes['gameData']['teams']['away']['name']
    awayId = gameRes['gameData']['teams']['away']['id']
    for play in gameRes['liveData']['plays']['allPlays']:
        result = play['result']
        event = result['event']
        eventTypeId = result['eventTypeId']
        secondaryType = '';
        if 'secondaryType' in result:
            secondaryType = result['secondaryType']
        penaltySeverity = ''
        if 'penaltySeverity' in result:
            penaltySeverity = result['penaltySeverity']
        penaltyMinutes = ''
        if 'penaltyMinutes' in result:
            penaltyMinutes = result['penaltyMinutes']
        eventIDx = play['about']['eventIdx']
        eventID = play['about']['eventId']
        period = play['about']['period']
        periodTime = play['about']['periodTime']
        player1ID = ''
        player1FullName = ''
        player1Type = ''
        player2ID = ''
        player2FullName = ''
        player2Type = ''
        player3ID = ''
        player3FullName = ''
        player3Type = ''
        player4ID = ''
        player4FullName = ''
        player4Type = ''
        teamID = ''
        teamName = ''
        if 'team' in play:
            teamID = play['team']['id']
            teamName = play['team']['name']
        if 'players' in play:
            player1ID = play['players'][0]['player']['id']
            player1FullName = play['players'][0]['player']['fullName']
            player1Type = play['players'][0]['playerType']
            if len(play['players']) > 1:  # 2 players involved
                player2ID = play['players'][1]['player']['id']
                player2FullName = play['players'][1]['player']['fullName']
                player2Type = play['players'][1]['playerType']
            if len(play['players']) > 2: # 3 players involved
                player3ID = play['players'][2]['player']['id']
                player3FullName = play['players'][2]['player']['fullName']
                player3Type = play['players'][2]['playerType']
            if len(play['players']) > 3: # 4 players involved
                player4ID = play['players'][3]['player']['id']
                player4FullName = play['players'][3]['player']['fullName']
                player4Type = play['players'][3]['playerType']

        dict1 = {
            "gameId" : gameID,
            "homeTeam" : homeTeam,
            "homeTeamId" : homeId,
            "awayTeam" : awayTeam,
            "awayTeamId" : awayId,
            "event" : event,
            "eventTypeId" : eventTypeId,
            "eventIDx" : eventIDx,
            "eventID" : eventID,
            "secondaryType" : secondaryType,
            "penaltySeverity" : penaltySeverity,
            "penaltyMinutes" : penaltyMinutes,
            "period" : period,
            "periodTime" : periodTime,
            "player1ID" : player1ID,
            "player1FullName" : player1FullName,
            "player1Type" : player1Type,
            "player2ID" : player2ID,
            "player2FullName" : player2FullName,
            "player2Type" : player2Type,
            "player3ID" : player3ID,
            "player3FullName" : player3FullName,
            "player3Type" : player3Type,
            "player4ID" : player4ID,
            "player4FullName" : player4FullName,
            "player4Type" : player4Type,
            "teamID" : teamID,
            "teamName" : teamName
        }
        events.append(dict1)
    return events
    

In [ ]:
def getFightGameData(gameID):
    events = []
    gameRes = nhl_request(f'game/{gameID}/feed/live')
    hasFight = False
    for penaltyPlay in gameRes['liveData']['plays']['penaltyPlays']:
        play = gameRes['liveData']['plays']['allPlays'][penaltyPlay]
        if play['result']['secondaryType'] == 'Fighting':
            hasFight = True
    if not hasFight:
#         print(f'game {gameID} has no fights')
        return events
    else:
        print(f'Found a fight in game {gameID}')
        homeTeam = gameRes['gameData']['teams']['home']['name']
        homeId = gameRes['gameData']['teams']['home']['id']
        awayTeam = gameRes['gameData']['teams']['away']['name']
        awayId = gameRes['gameData']['teams']['away']['id']
        for play in gameRes['liveData']['plays']['allPlays']:
            result = play['result']
            event = result['event']
            eventTypeId = result['eventTypeId']
            secondaryType = '';
            if 'secondaryType' in result:
                secondaryType = result['secondaryType']
            penaltySeverity = ''
            if 'penaltySeverity' in result:
                penaltySeverity = result['penaltySeverity']
            penaltyMinutes = ''
            if 'penaltyMinutes' in result:
                penaltyMinutes = result['penaltyMinutes']
            eventIDx = play['about']['eventIdx']
            eventID = play['about']['eventId']
            period = play['about']['period']
            periodTime = play['about']['periodTime']
            player1ID = ''
            player1FullName = ''
            player1Type = ''
            player2ID = ''
            player2FullName = ''
            player2Type = ''
            player3ID = ''
            player3FullName = ''
            player3Type = ''
            player4ID = ''
            player4FullName = ''
            player4Type = ''
            teamID = ''
            teamName = ''
            if 'team' in play:
                teamID = play['team']['id']
                teamName = play['team']['name']
            if 'players' in play:
                player1ID = play['players'][0]['player']['id']
                player1FullName = play['players'][0]['player']['fullName']
                player1Type = play['players'][0]['playerType']
                if len(play['players']) > 1:  # 2 players involved
                    player2ID = play['players'][1]['player']['id']
                    player2FullName = play['players'][1]['player']['fullName']
                    player2Type = play['players'][1]['playerType']
                if len(play['players']) > 2: # 3 players involved
                    player3ID = play['players'][2]['player']['id']
                    player3FullName = play['players'][2]['player']['fullName']
                    player3Type = play['players'][2]['playerType']
                if len(play['players']) > 3: # 4 players involved
                    player4ID = play['players'][3]['player']['id']
                    player4FullName = play['players'][3]['player']['fullName']
                    player4Type = play['players'][3]['playerType']

            dict1 = {
                "gameId" : gameID,
                "homeTeam" : homeTeam,
                "homeTeamId" : homeId,
                "awayTeam" : awayTeam,
                "awayTeamId" : awayId,
                "event" : event,
                "eventTypeId" : eventTypeId,
                "eventIDx" : eventIDx,
                "eventID" : eventID,
                "secondaryType" : secondaryType,
                "penaltySeverity" : penaltySeverity,
                "penaltyMinutes" : penaltyMinutes,
                "period" : period,
                "periodTime" : periodTime,
                "player1ID" : player1ID,
                "player1FullName" : player1FullName,
                "player1Type" : player1Type,
                "player2ID" : player2ID,
                "player2FullName" : player2FullName,
                "player2Type" : player2Type,
                "player3ID" : player3ID,
                "player3FullName" : player3FullName,
                "player3Type" : player3Type,
                "player4ID" : player4ID,
                "player4FullName" : player4FullName,
                "player4Type" : player4Type,
                "teamID" : teamID,
                "teamName" : teamName
            }
            events.append(dict1)
        return events
    

In [ ]:
def getFightGameDataCSV(season):
    gameIDs = getSeasonGameIDs(season)
    data = []
    for gameID in gameIDs:
        data = data + getFightGameData(gameID)
    df = pd.DataFrame(data)
    df.to_csv(f'AllFightGameData-{season}.csv', index=False)
    print(f'Data gathering for {season} complete')

In [ ]:
# Get fight game data for each year
getFightGameDataCSV(20152016)
getFightGameDataCSV(20162017)
getFightGameDataCSV(20172018)
getFightGameDataCSV(20182019)
getFightGameDataCSV(20192020)
getFightGameDataCSV(20202021)